In [ ]:
import numpy as np
import cupy as cp

In [ ]:
x_gpu = cp.array([[1, 2, 3], [2, 4, 6], [4, 8, 12]])
x_gpu

In [ ]:
cp.fft.rfft2(x_gpu)

In [ ]:
cp.fft.fft2(x_gpu.T)

In [ ]:
cp.fft.fftfreq(3)

In [ ]:
cp.fft.rfftfreq(3)

In [ ]:
import cupy as cp
from cupyx.scipy.fft import get_fft_plan

a = cp.random.random((4, 400, 400)).astype(cp.complex64)
plan = get_fft_plan(a, axes=(1, 2), value_type='C2C')  # for batched, C2C, 2D transform

In [ ]:
with plan:
    # the arguments must match those used when generating the plan
    out = cp.fft.fft2(a, axes=(1, 2))

In [ ]:
out.shape

In [ ]:
with plan:
    %timeit cp.fft.fft2(a, axes=(1, 2))

In [ ]:
from sympy import *

In [ ]:
# define symbols
x = Symbol('x')
y = cos(x) * exp(-x**2 / 25)
f = lambdify(x, y, 'numpy')
dy_x = diff(y, x)
simplify(dy_x)
df = lambdify(x, dy_x, 'numpy')

In [ ]:
n = 1024
L = 30
dx = L/n
X = np.arange(-L/2, L/2, dx, dtype=np.complex128)
k = np.fft.fftshift((2*np.pi/L) * np.arange(-n/2, n/2))

In [ ]:
u = f(X)
du = df(X)

In [ ]:
u_cp = cp.asarray(u)
k_cp = cp.asarray(k)

In [ ]:
def fft_derivative(u_cp, k_cp):
    uhat = cp.fft.fft(u_cp)
    duhat = k_cp * uhat * (1.j)
    duFFT_cp = cp.fft.ifft(duhat).real
    return duFFT_cp

In [ ]:
@cp.fuse()
def multiply(x, y):
    return (x*y)

In [ ]:
ik_cp = 1.j * k_cp

In [ ]:
def fft_derivative2(u_cp, ik_cp):
    uhat = cp.fft.fft(u_cp)
    duhat = multiply(ik_cp, uhat)
    duFFT_cp = cp.fft.ifft(duhat).real
    return duFFT_cp

In [ ]:
duFFT_cp = fft_derivative(u_cp, k_cp)

In [ ]:
duFFT_cp2 = fft_derivative2(u_cp, ik_cp)

In [ ]:
%timeit -n10000 fft_derivative(u_cp, k_cp)

In [ ]:
%timeit -n10000 fft_derivative2(u_cp, ik_cp), 10000

In [ ]:
# plot A with plotly
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=X.real, y=u_cp.get().real, mode='lines', name='real'))
# fig.add_trace(go.Scatter(x=X, y=A.imag, mode='lines', name='imag'))

# title
fig.update_layout(title=f'f(x) = {y}', xaxis_title='X', yaxis_title='A')
fig.show()

In [ ]:
# plot A with plotly
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=X.real, y=du.real, mode='lines', name='analytic'))
fig.add_trace(go.Scatter(x=X.real, y=duFFT_cp.get().real, mode='lines', name='FFT'))
# fig.add_trace(go.Scatter(x=X, y=A.imag, mode='lines', name='imag'))

# title
fig.update_layout(title=f'f\'(x) = {dy_x}', xaxis_title='X', yaxis_title='A')
fig.show()

In [ ]:
# plot A with plotly
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=X.real, y=du.real, mode='lines', name='analytic'))
fig.add_trace(go.Scatter(x=X.real, y=duFFT_cp2.get().real, mode='lines', name='FFT'))
# fig.add_trace(go.Scatter(x=X, y=A.imag, mode='lines', name='imag'))

# title
fig.update_layout(title=f'f\'(x) = {dy_x}', xaxis_title='X', yaxis_title='A')
fig.show()

In [ ]:
squared_diff = cp.ElementwiseKernel(
   'float32 x, float32 y',
   'float32 z',
   'z = (x - y) * (x - y)',
   'squared_diff')

In [ ]:
x = cp.arange(10, dtype=np.float32).reshape(2, 5)
y = cp.arange(5, dtype=np.float32)
squared_diff(x, y)

In [ ]:
x

In [ ]:
squared_diff_generic = cp.ElementwiseKernel(
    'T x, T y',
    'T z',
    'z = (x - y) * (x - y)',
    'squared_diff_generic')

In [ ]:
x = cp.arange(10, dtype=np.complex128).reshape(2, 5)+1j*(5-cp.arange(5, dtype=np.complex128))
y = cp.arange(5, dtype=np.complex128)
squared_diff_generic(x, y)

In [ ]:
x

In [ ]:
y

In [ ]:
(x-y)**2

In [ ]:
@cp.fuse()
def squared_diff(x, y):
    return (x - y) * (x - y)

In [ ]:
x_cp = cp.arange(10)
y_cp = cp.arange(10)[::-1]
squared_diff(x_cp, y_cp)

In [ ]:
x_np = np.arange(10)
y_np = np.arange(10)[::-1]
squared_diff(x_np, y_np)

In [ ]:
%timeit squared_diff(x_cp, y_cp)